In [13]:
import torch
import gdown
import os
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch.nn.functional as F

# ==========================================================================
# [설정] 두 모델의 Google Drive ID를 각각 입력하세요!
# ==========================================================================
# 1. 팀플 분류 모델 (Is_Teamplay) - 기존 model.pt ID
teamplay_file_id = '1hcl250N4eFpdCpxIZlJNLQAv5FJpUvRX'

# 2. 과제 부담 모델 (Burden_Score) - 방금 업로드한 burden_model.pt ID
burden_file_id   = '1fCQ8Qr_GxJtcqAn7l91_Bf64GdDfRyyC'  # <--- 여기에 새 ID 붙여넣기

# ==========================================================================

# 파일 다운로드 함수
def download_file(file_id, filename):
    url = f'https://drive.google.com/uc?id={file_id}'
    if os.path.exists(filename):
        os.remove(filename) # 강제 재다운로드
    print(f"📥 {filename} 다운로드 중... (ID: {file_id})")
    gdown.download(url, filename, quiet=False)

# 두 모델 다운로드 실행
download_file(teamplay_file_id, 'model.pt')
download_file(burden_file_id, 'burden_model.pt')

# --------------------------------------------------------------------------
# 모델 로드 함수 (분류용 vs 회귀용 구분)
# --------------------------------------------------------------------------
MODEL_NAME = "monologg/distilkobert"
MAX_LEN = 128
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def load_models():
    print("\n⏳ 모델 2개 로딩 중...")
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)

    # 1. 팀플 분류 모델 로드
    model_cls = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2, trust_remote_code=True)
    try:
        model_cls.load_state_dict(torch.load('model.pt', map_location=device, weights_only=False))
    except:
        model_cls.load_state_dict(torch.load('model.pt', map_location=device))
    model_cls.to(device).eval()

    # 2. 과제 부담(회귀) 모델 로드 (num_labels=1)
    model_reg = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=1, trust_remote_code=True)
    try:
        model_reg.load_state_dict(torch.load('burden_model.pt', map_location=device, weights_only=False))
    except:
        model_reg.load_state_dict(torch.load('burden_model.pt', map_location=device))
    model_reg.to(device).eval()

    print("✅ 모든 모델 로드 완료!")
    return tokenizer, model_cls, model_reg

# --------------------------------------------------------------------------
# 통합 추론 함수
# --------------------------------------------------------------------------
def analyze_syllabus(tokenizer, model_cls, model_reg, text):
    inputs = tokenizer(
        text, return_tensors='pt', max_length=MAX_LEN, truncation=True, padding='max_length'
    )
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    with torch.no_grad():
        # 1. 팀플 예측
        out_cls = model_cls(input_ids=input_ids, attention_mask=attention_mask)
        probs = F.softmax(out_cls.logits, dim=1)
        prob_team = probs[0][1].item() * 100
        is_teamplay = torch.argmax(out_cls.logits, dim=1).item()

        # 2. 과제 부담 예측 (0.0 ~ 1.0 -> 0 ~ 100점으로 변환)
        out_reg = model_reg(input_ids=input_ids, attention_mask=attention_mask)
        burden_score = out_reg.logits.item() * 100

    return is_teamplay, prob_team, burden_score

# --------------------------------------------------------------------------
# 시나리오 테스트
# --------------------------------------------------------------------------
try:
    if os.path.exists('model.pt') and os.path.exists('burden_model.pt'):
        tokenizer, model_cls, model_reg = load_models()

        print("\n" + "="*60)
        print("🎓 HAT-N 최종 분석 (팀플 유무 & 과제 부담 예측)")
        print("="*60)

        # [Scenario 1] 팀플 O, 과제 많음 (매운맛)
        scenario_1 = """
        과목명: 창업캡스톤디자인 / 수업개요: 본 수업은 100% 팀 프로젝트 중심으로 진행됩니다.
        학생들은 팀을 구성하여 한 학기 동안 팀 프로젝트를 수행하며, 매주 팀 미팅과 팀 보고서 작성이 필수입니다.
        중간/기말고사 없이 오직 팀 프로젝트 결과물로만 성적을 평가합니다.
        팀원 간의 협력이 무엇보다 중요하며, 개인 과제는 없습니다.
        """

        # [Scenario 2] 팀플 X, 과제 보통
        scenario_2 = """
        과목명: 서양철학사 / 수업개요: 고대부터 현대까지 서양 철학의 흐름을 학습합니다.
        수업은 교수님의 강의 위주로 진행되며, 별도의 조별 모임은 없습니다.
        평가는 중간고사 40%, 기말고사 40%, 그리고 출석 20%로 이루어집니다.
        과제는 학기 중 1회의 개인 독후감 제출이 있습니다.
        """

        # [Scenario 3] 팀플 X, 과제 매우 적음 (꿀강)
        scenario_3 = """
        과목명: 영화 감상과 비평 / 수업개요: 매주 영화를 감상하고 자유롭게 감상평을 나누는 수업입니다.
        중간고사와 기말고사는 없으며, 수업 참여도와 간단한 감상문 1회로 성적을 평가합니다.
        조별 과제나 발표는 일절 없습니다. 부담 없이 영화를 즐기시면 됩니다.
        """

        scenarios = [scenario_1, scenario_2, scenario_3]

        for i, text in enumerate(scenarios):
            is_team, prob_team, burden = analyze_syllabus(tokenizer, model_cls, model_reg, text)

            team_msg = "🚨 팀플 있음" if is_team == 1 else "🍀 팀플 없음"

            # 부담 점수 해석
            if burden > 50: burden_msg = "매우 높음 (수강 주의)"
            elif burden > 20: burden_msg = "보통"
            else: burden_msg = "매우 낮음 (꿀강)"

            print(f"\n[Scenario {i+1}]")
            print(f"📄 내용: {text.strip().split('/')[1][:40]}...")
            print(f"🤖 팀플 예측: {team_msg} (확률: {prob_team:.1f}%)")
            print(f"📉 과제 부담: {burden:.1f}점 -> {burden_msg}")
            print("-" * 50)

    else:
        print("❌ 모델 파일 다운로드 실패. Google Drive ID를 확인하세요.")

except Exception as e:
    print(f"❌ 오류 발생: {e}")

📥 model.pt 다운로드 중... (ID: 1hcl250N4eFpdCpxIZlJNLQAv5FJpUvRX)


Downloading...
From (original): https://drive.google.com/uc?id=1hcl250N4eFpdCpxIZlJNLQAv5FJpUvRX
From (redirected): https://drive.google.com/uc?id=1hcl250N4eFpdCpxIZlJNLQAv5FJpUvRX&confirm=t&uuid=e10f40bd-883a-43ad-9724-d2fa5643bf84
To: /content/model.pt
100%|██████████| 114M/114M [00:01<00:00, 98.6MB/s]


📥 burden_model.pt 다운로드 중... (ID: 1fCQ8Qr_GxJtcqAn7l91_Bf64GdDfRyyC)


Downloading...
From (original): https://drive.google.com/uc?id=1fCQ8Qr_GxJtcqAn7l91_Bf64GdDfRyyC
From (redirected): https://drive.google.com/uc?id=1fCQ8Qr_GxJtcqAn7l91_Bf64GdDfRyyC&confirm=t&uuid=cc4d3a15-7140-46e9-af26-dbe61c767966
To: /content/burden_model.pt
100%|██████████| 114M/114M [00:01<00:00, 74.8MB/s]



⏳ 모델 2개 로딩 중...


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at monologg/distilkobert and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at monologg/distilkobert and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ 모든 모델 로드 완료!

🎓 HAT-N 최종 분석 (팀플 유무 & 과제 부담 예측)

[Scenario 1]
📄 내용:  수업개요: 본 수업은 100% 팀 프로젝트 중심으로 진행됩니다. 
  ...
🤖 팀플 예측: 🚨 팀플 있음 (확률: 52.7%)
📉 과제 부담: 18.7점 -> 매우 낮음 (꿀강)
--------------------------------------------------

[Scenario 2]
📄 내용:  수업개요: 고대부터 현대까지 서양 철학의 흐름을 학습합니다.
     ...
🤖 팀플 예측: 🍀 팀플 없음 (확률: 46.1%)
📉 과제 부담: 13.1점 -> 매우 낮음 (꿀강)
--------------------------------------------------

[Scenario 3]
📄 내용:  수업개요: 매주 영화를 감상하고 자유롭게 감상평을 나누는 수업입니다.
...
🤖 팀플 예측: 🍀 팀플 없음 (확률: 40.1%)
📉 과제 부담: 19.8점 -> 매우 낮음 (꿀강)
--------------------------------------------------
